In [6]:
from dataset import AarizDataset

path = "Aariz Dataset"
dataset = AarizDataset( path ,"TRAIN")

In [7]:
image, landmarks, cvm = dataset[0]

In [8]:
dataset.get_image("cks2ip8fq2a0j0yufdfssbc09.png")
dataset.get_landmarks("cks2ip8fq2a0j0yufdfssbc09.json")
dataset.get_cvm_stage("cks2ip8fq2a0j0yufdfssbc09.json")

array([[1., 0., 0., 0., 0., 0.]], dtype=float32)